# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [3]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

# KT's bits:
# help(response)


200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [4]:
earthquake_json = response.json()
earthquake_json.keys()


dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [5]:
earthquake_json['metadata']
# earthquake_json['bbox']


{'generated': 1652436805000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-04-12&endtime=2022-05-12',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 9287}

Each element in the JSON array `features` is a row of data for our dataframe.

In [6]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [7]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.92999995,
  'place': '9 km E of P?hala, Hawaii',
  'time': 1652313293090,
  'updated': 1652313477280,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv73009882',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv73009882&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 57,
  'net': 'hv',
  'code': '73009882',
  'ids': ',hv73009882,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 30,
  'dmin': None,
  'rms': 0.109999999,
  'gap': 177,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 1.9 - 9 km E of P?hala, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.393005371094, 19.1873340606689, 30.5499992370605]},
 'id': 'hv73009882'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [8]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.93,"9 km E of P?hala, Hawaii",1652313293090,1652313477280,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73009882,",",hv,",",origin,phase-data,",30.0,NaN,0.11,177.0,md,earthquake,"M 1.9 - 9 km E of P?hala, Hawaii"
1,0.76,"4km SW of Anza, CA",1652313179940,1652313674970,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci40016007,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",13.0,0.05144,0.28,115.0,ml,earthquake,"M 0.8 - 4km SW of Anza, CA"
2,0.90,"29 km ESE of Minto, Alaska",1652312996378,1652313733139,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022611e21j,",",ak,",",origin,phase-data,",NaN,NaN,0.75,NaN,ml,earthquake,"M 0.9 - 29 km ESE of Minto, Alaska"
3,1.40,"42 km NW of Skwentna, Alaska",1652312708989,1652312883168,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022611d3hv,",",ak,",",origin,phase-data,",NaN,NaN,0.71,NaN,ml,earthquake,"M 1.4 - 42 km NW of Skwentna, Alaska"
4,1.27,"5km NW of Mira Loma, CA",1652312643350,1652314088569,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,1.0,2.2,NaN,...,",ci40015991,",",ci,",",dyfi,nearby-cities,origin,phase-data,scitech-...",29.0,0.01621,0.20,59.0,ml,earthquake,"M 1.3 - 5km NW of Mira Loma, CA"


### (Optional) Write Data to CSV

In [9]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>